<font size="6"><b> Outils numériques pour la présentation de TIPE </b></font>

Pour traiter et présenter les données numériques recueillies lors de vos expériences, il est fortement recommandé d'utiliser le langage Python. La suite présente ainsi comment :
- représenter graphiquement des points expérimentaux ;
- créer une courbe d'ajustement ;
- estimer et représenter graphiquement l'incertitude sur chaque mesure ;
- résoudre une équation différentielle et représenter graphiquement les résultats.

> Pour le bon fonctionnement des représentations graphiques dans ce notebook, exécuter simplement la ligne suivante :

In [ ]:
%matplotlib notebook

# Représenter graphiquement des points expérimentaux
## À partir de données saisies manuellement
Si vos données sont en petit nombre, on peut procéder tout simplement ainsi :

In [ ]:
import matplotlib.pyplot as plt

# Valeurs à traiter
liste_T=[92.6,79.9,71.5,65.2,60.2,55.1,51.2,48,45.5,42.8,40.3,38.3,36.5,34.1,32.8,27.5,22.5,20.3,18.9]
liste_t=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,90,120,150,180]

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_T,'r+')
plt.title('Représentation de T(t)')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.legend(('T'))
plt.savefig('courbe.png')     # Pour éventuellement sauvegarder la figure obtenue
plt.show()

## À partir de données présentes dans un fichier
Si vos données expérimentales sont présentes dans un fichier externe (par exemple après exportation depuis un logiciel), type `TXT` ou `CSV`, il faut savoir comment celles-ci sont disposées dans ce fichier, notamment :
- Y a-t-il une première ligne de titre ?
- Quel est le séparateur entre les données ?

Pour le savoir, vous devez ouvrir le fichier en question, soit avec un simple éditeur de texte présent de base sur tous les sytèmes d'exploitation, ou avec un tableur type LibreOffice. Le résultat avec un éditeur de texte donne par exemple :

<img src="donneestxt.png" width="250"/>

On constate ainsi que :
- une première ligne de titre existe ;
- le séparateur des données est le symbole `;`.

L'ouverture d'un fichier `CSV` avec LibroOffice génère l'ouverture d'une fenêtre :

<img src="donneescsv.png" width="300"/>

Le logiciel LibreOffice a ici détecté le type de séparateur, en pousuivant vous avez accès aux données dans le tableur.

Ainsi on peut désormais récupérer les données dans un programme Python :

In [ ]:
import matplotlib.pyplot as plt

nomfichier='donnees.csv'                         # Bien préciser le nom du fichier

# Importation des données
liste_T=[]
liste_t=[]
with open(nomfichier, 'r') as fichier:
    fichier.readline()
    for ligne in fichier:
        ligne_lue=ligne.split(';')                           # Le séparateur entre les données est ';'
        liste_t.append(float(ligne_lue[0].replace(',','.'))) # On remplace le séparateur décimal ',' par '.' pour utiliser les données en Python
        liste_T.append(float(ligne_lue[1].replace(',','.'))) # On remplace le séparateur décimal ',' par '.' pour utiliser les données en Python

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_T,'r+')
plt.title('Représentation de T(t)')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.legend(('T'))
plt.savefig('courbe.png')
plt.show()

> **Attention de bien préciser le nom du fichier**, l'essentiel des soucis dans l'utilisation de ce programme vient d'un nom incorrect de fichier. Pour éviter cela on peut améliorer le programme en lui faisant ouvrir une boîte de dialogue pour sélectionner le fichier à ouvrir :

In [ ]:
import matplotlib.pyplot as plt
from tkinter.filedialog import askopenfilename

nomfichier=askopenfilename(filetypes=(('Fichier CSV','*.csv'),('Fichier TXT','*.txt'),('Tous les fichiers','*.*')),title = "Choisir un fichier")

# Importation des données
liste_T=[]
liste_t=[]
with open(nomfichier, 'r') as fichier:
    fichier.readline()
    for ligne in fichier:
        ligne_lue=ligne.split(';')                           # Le séparateur entre les données est ';'
        liste_t.append(float(ligne_lue[0].replace(',','.'))) # On remplace le séparateur décimal ',' par '.' pour utiliser les données en Python
        liste_T.append(float(ligne_lue[1].replace(',','.'))) # On remplace le séparateur décimal ',' par '.' pour utiliser les données en Python

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_T,'r+')
plt.title('Représentation de T(t)')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.legend(('T'))
plt.savefig('courbe.png')
plt.show()

## À parir de données issues d'un fichier audio
Dans le cas d'un fichier audio, les données peuvent être récupérées automatiquement à l'aide de divers modules, par exemple :
> Attention, utilisez des fichiers au format `WAV` qui ne possède aucune compression des données.

In [ ]:
import scipy as sp
import scipy.io.wavfile as spwf
import matplotlib.pyplot as plt

fichier='son_corde_guitare.wav'

fe=spwf.read(fichier)[0]              # Récupération de la fréquence d'échantillonnage de l'enregistrement
k_tot=len(spwf.read(fichier)[1])      # Recuperation du nombre de mesures
t_max=k_tot/fe

liste_t=sp.linspace(0,t_max,k_tot)    # Création de la liste des temps
liste_a=spwf.read(fichier)[1]         # Récupération des mesures

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_a)
plt.xlabel('t (s)')
plt.ylabel('Amplitude')
plt.savefig('son.png')
plt.show()

# Créer une courbe d'ajustement
Pour créer une courbe d'ajustement, il est pratique d'utiliser la fonction `curve_fit` de la bibliothèque `scipy`. Par exemple pour réaliser un ajustement exponentiel :

In [ ]:
import matplotlib.pyplot as plt # Pour les graphes'
import scipy.optimize as spo    # Pour la fonction 'curve_fit'
import numpy as np              # Pour les fonctions 'exp' et 'linspace'

nomfichier='donnees.csv'

# Importation des données
liste_t=[]
liste_T=[]
with open(nomfichier, 'r') as fichier:
    fichier.readline()
    for ligne in fichier:
        ligne_lue=ligne.split(';')
        liste_t.append(float(ligne_lue[0].replace(',','.')))
        liste_T.append(float(ligne_lue[1].replace(',','.')))

# Ajustement selon une fonction d'ajustement définie
def f_ajust(x,a,b,c):
    return(a*np.exp(-b*x)+c)

popt,pcov=spo.curve_fit(f_ajust,liste_t,liste_T)                       # 'pcov' est la matrice de covariance

print('Équation a*exp(-b*x)+c : a=',popt[0],'b=',popt[1],'c=',popt[2]) # On peut utiliser round(nombre,nb_decimales) pour régler l'affichage

# Création des listes de la fonction ajustée
liste_Tajust=[]
liste_tajust=np.linspace(liste_t[0],liste_t[-1],200)                   # Liste des valeurs des abscisses pour la fonction ajustée
for t in liste_tajust:
    liste_Tajust.append(f_ajust(t,*popt))                              # '*' indique que 'popt' est une liste de paramètres

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_T,'r+')
plt.plot(liste_tajust,liste_Tajust,'b-')
plt.title('Représentation de T(t)')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.legend(('T','Tajustée'))
plt.savefig('courbe_ajustement.png')
plt.show()

# Estimer et représenter graphiquement l'incertitude-type sur chaque mesure
Chaque mesure possède une incertitude-type qu'il faut évaluer.
> Ne pas confondre l'incertitude-type associée à une mesure avec la résolution de l'appareil de mesure :
> - l'incertitude-type est un ordre de grandeur du domaine de variation de la grandeur (elle variera sur quelques incertitudes-type (voir [Incertitudes expérimentales](https://mybinder.org/v2/gh/peleroy/travauxpratiques/HEAD?labpath=incertitudes_de_mesure.ipynb)) ;
> - la résolution de l'appareil de mesure est la plus petite variation de mesure qu'il peut déceler.
>
> Ainsi par exemple une règle graduée au mm près a une précision de 1 mm. Mais l'incertitude associée à une mesure avec cette règle est plus faible puisque l'on est systématiquement proche d'une graduation plutôt qu'une autre. On peut l'estimer à $0,5/\sqrt 3$ mm.

L'incertitude-type associée à chaque mesure est représentée graphiquement par des barres d'incertitude-type. Pour chaque mesure on peut avoir une double incertitude-type : sur l'ordonnée et sur l'abscisse.

Par exemple si l'incertitude-type est la même sur chaque mesure :

In [ ]:
import matplotlib.pyplot as plt

# Listes des valeurs à traiter
liste_T=[92.6,79.9,71.5,65.2,60.2,55.1,51.2,48,45.5,42.8,40.3,38.3,36.5,34.1,32.8,27.5,22.5,20.3,18.9]
liste_t=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,90,120,150,180]

# Incertitudes sur chaque mesure
incert_x=2.0 # Incertitude-type sur l'abscisse
incert_y=2.5 # Incertitude-type sur l'ordonnée

# Représentation graphique
plt.figure()
plt.errorbar(liste_t,liste_T,incert_y,incert_x,fmt='r+',ecolor='blue',capsize=2) # Cas d'une incertitude-type constante

plt.title('Représentation de T(t)')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.legend(('T'))
plt.savefig('courbe_barres_incertitudes.png')
plt.show()

Si l'incertitude est différente sur chaque mesure, on utilise des listes :

In [ ]:
import matplotlib.pyplot as plt

# Listes des valeurs à traiter
liste_T=[92.6,79.9,71.5,65.2,60.2,55.1,51.2,48,45.5,42.8,40.3,38.3,36.5,34.1,32.8,27.5,22.5,20.3,18.9]
liste_t=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,90,120,150,180]

# Si les incertitudes-type ne sont pas les même sur chaque mesure, créer des listes
liste_incert_x=[2.0]*len(liste_t)
liste_incert_y=[T*10/100 for T in liste_T]

# Représentation graphique
plt.figure()
plt.errorbar(liste_t,liste_T,liste_incert_y,liste_incert_x,fmt='r+',ecolor='blue',capsize=2) # Cas d'une incertitude-type variable

plt.title('Représentation de T(t)')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.legend(('T'))
plt.savefig('courbe_barres_incertitudes.png')
plt.show()

# Résoudre une équation différentielle et représenter graphiquement les résultats
## Méthode d'Euler
La méthode d'Euler est la méthode de base de toutes les méthodes de résolution numérique d'une équation différentielle. Le principe est de discrétiser les relations différentielles, par exemple :
$$
\dfrac{\mathrm{d} u}{\mathrm{d}t}+a \cdot u^2=b
$$

devient par discrétisation, selon le principe que $\dfrac{\mathrm{d} u}{\mathrm{d}t}$ devient $\dfrac{u_{n+1}-u_n}{\mathrm{d}t}$ :

$$
\boxed{u_{n+1}=u_n+\mathrm{d}t \cdot (-a \cdot u_n^2+b)}
$$

Pour une équation différentielle d'ordre 2 :
$$
\dfrac{\mathrm{d}^2 u}{\mathrm{d}t^2}+a \cdot \dfrac{\mathrm{d} u}{\mathrm{d}t}+b \cdot u^2=c
$$

il est commode d'introduire la grandeur dérivée $\dfrac{\mathrm{d} u}{\mathrm{d}t}$, ce qui revient alors à résoudre numériquement deux équations, ce qui donne par discrétisation :

$$
\boxed{u_{n+1}=u_n+\mathrm{d}t \cdot \left. \dfrac{\mathrm{d} u}{\mathrm{d}t} \right|_n}
$$

et

$$
\boxed{\left. \dfrac{\mathrm{d} u}{\mathrm{d}t} \right|_{n+1}=\left. \dfrac{\mathrm{d} u}{\mathrm{d}t} \right|_n+\mathrm{d}t \cdot \left( -a \cdot \left. \dfrac{\mathrm{d} u}{\mathrm{d}t} \right|_n -b \cdot u_n^2+c \right)}
$$

Cela donne pour la première équation :

In [ ]:
import matplotlib.pyplot as plt

### Résolution de l'équation différentielle du/dt+a.u^2=b
# Définition des constantes
a=7.1
b=2.4
u0=0 # Valeur initiale de u

# Paramètres de la résolution numérique
N=2000 # Nombre de points de la résolution numérique
duree=1.2 # Durée totale
dt=duree/(N-1) # Pas de la résolution numérique

# Définition des listes
liste_u=[u0]
liste_t=[0]

# Résolution par la méthode d'Euler (simple discrétisation)
for i in range(N-1):
    liste_u.append(liste_u[i]+dt*(-a*liste_u[i]**2+b))
    liste_t.append(liste_t[i]+dt)

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_u,'r-')
plt.title('Représentation de u(t)')
plt.xlabel('t (s)')
plt.ylabel('u')
plt.legend(('u'))
plt.savefig('equa_diff_euler1o.png')
plt.show()

Et pour la seconde :

In [ ]:
import matplotlib.pyplot as plt

### Résolution de l'équation différentielle d^2u/dt^2+a.du/dt+b.u^2=c
# Définition des constantes
a=7.1
b=2.4
c=2.1
u0=0 # Valeur initiale de u
dudt0=2 # Valeur initiale de du/dt

# Paramètres de la résolution numérique
N=2000 # Nombre de points de la résolution numérique
duree=1.2 # Durée totale
dt=duree/(N-1) # Pas de la résolution numérique

# Définition des listes et de la valeur initiale de la dérivée du/dt
liste_u=[u0]
liste_t=[0]
dudt=dudt0

# Résolution par la méthode d'Euler (simple discrétisation)
for i in range(N-1):
    dudt=dudt+dt*(-a*dudt-b*liste_u[i]**2+c)
    liste_u.append(liste_u[i]+dt*dudt)
    liste_t.append(liste_t[i]+dt)

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_u,'r-')
plt.title('Représentation de u(t)')
plt.xlabel('t (s)')
plt.ylabel('u')
plt.legend(('u'))
plt.savefig('equa_diff_euler2o.png')
plt.show()

## Utilisation de la bibliothèque `scipy.integrate`
On peut faire la même chose avec une fonction intégrée à la bibliothèque `scipy`, qui utilise des méthodes bien plus évoluées que la méthode d'Euler. Par exemple pour la première équation :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as spi

### Résolution de l'équation différentielle du/dt+a.u^2=b
# Définition des constantes
a=7.1
b=2.4
u0=0 # Valeur initiale de u

# Paramètres de la résolution numérique
N=2000 # Nombre de points de la résolution numérique
duree=1.2 # Durée totale

# Définition de la fonction f telle que du/dt=f(u,t)
def f(u,t):
    return(-a*u**2+b)

# Définition de la liste des temps
liste_t=np.linspace(0,duree,N)

# Résolution numérique
sol=spi.odeint(f,u0,liste_t)

# Transformation de la matrice solution en liste
liste_u=list(sol[:,0])

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_u,'r-')
plt.title('Représentation de u(t)')
plt.xlabel('t (s)')
plt.ylabel('u')
plt.legend(('u'))
plt.savefig('equa_diff_odeint1o.png')
plt.show()

Et pour la seconde :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.integrate as spi

### Résolution de l'équation différentielle d^2u/dt^2+a.du/dt+b.u^2=c
# Définition des constantes
a=7.1
b=2.4
c=2.1
u0=0 # Valeur initiale de u
dudt0=2 # Valeur initiale de du/dt

# Paramètres de la résolution numérique
N=2000 # Nombre de points de la résolution numérique
duree=1.2 # Durée totale

# Définition de la fonction f telle que, en introduisant le vecteur U=(du/dt,u), on a dU/dt=f(U,t)
def f(U,t):
    dudt,u=U
    return([-a*dudt-b*u**2+c,dudt])

# Définition de la liste des temps
liste_t=np.linspace(0,duree,N)

# Résolution numérique
sol=spi.odeint(f,[dudt0,u0],liste_t)

# Transformation de la matrice solution en liste
liste_u=list(sol[:,1])

# Représentation graphique
plt.figure()
plt.plot(liste_t,liste_u,'r-')
plt.title('Représentation de u(t)')
plt.xlabel('t (s)')
plt.ylabel('u')
plt.legend(('u'))
plt.savefig('equa_diff_odeint2o.png')
plt.show()